In [14]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

import datetime
import csv
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

In [15]:
MOTOR = "Nabla"
PATH = f"../dataset/{MOTOR}/"
TRAIN_FILE = "_all_scaled_train.csv"
TEST_FILE = "_all_scaled_test.csv"

train_data = pd.DataFrame()

train_data = pd.concat([train_data, pd.read_csv(f'{PATH}idiq{TRAIN_FILE}').drop(columns = "Unnamed: 0")], axis = 1)
train_data['speed'] = pd.read_csv(f'{PATH}speed{TRAIN_FILE}')['N']
train_data = pd.concat([train_data, pd.read_csv(f'{PATH}xgeom{TRAIN_FILE}').drop(columns = "Unnamed: 0")], axis = 1)
train_data['hysteresis'] = pd.read_csv(f'{PATH}hysteresis{TRAIN_FILE}')['total']
train_data['joule'] = pd.read_csv(f'{PATH}joule{TRAIN_FILE}')['total']

test_data = pd.DataFrame()

test_data = pd.concat([test_data, pd.read_csv(f'{PATH}idiq{TEST_FILE}').drop(columns = "Unnamed: 0")], axis = 1)
test_data['speed'] = pd.read_csv(f'{PATH}speed{TEST_FILE}')['N']
test_data = pd.concat([test_data, pd.read_csv(f'{PATH}xgeom{TEST_FILE}').drop(columns = "Unnamed: 0")], axis = 1)
test_data['hysteresis'] = pd.read_csv(f'{PATH}hysteresis{TEST_FILE}')['total']
test_data['joule'] = pd.read_csv(f'{PATH}joule{TEST_FILE}')['total']

In [16]:
class Autoencoder(nn.Module):
    
    def __init__(self, features_dim, num_out, neurons = 5, layers = 1):
        super().__init__()

        modules = []
        
        modules.append(nn.Linear(features_dim, neurons))
        modules.append(nn.ReLU())
        for i in range(layers):
            modules.append(nn.Linear(neurons, neurons))
            modules.append(nn.ReLU())
        modules.append(nn.Linear(neurons, num_out))
        
        self.encoder = nn.Sequential(*modules)

        modules = []

        modules.append(nn.Linear(num_out, neurons))
        modules.append(nn.ReLU())
        for i in range(layers):
            modules.append(nn.Linear(neurons, neurons))
            modules.append(nn.ReLU())
        modules.append(nn.Linear(neurons, features_dim))

        self.decoder = nn.Sequential(*modules)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def encode(self, x):
        return self.encode(x)

    def decode(self, x):
        return self.decoder(x)

In [17]:
class MotorDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [18]:
def register_csv(contents, info, MOTOR):
    new_row = pd.DataFrame([contents], columns = info.columns)
    info = pd.concat([info, new_row])
    info.to_csv(f'./data/motor_{MOTOR}_info.csv')
    return info

In [19]:
target = ['hysteresis', 'joule']

train_dataset = MotorDataset(train_data.drop(columns = target), train_data[target])
test_dataset = MotorDataset(test_data.drop(columns = target), test_data[target])

BATCH_SIZE = 256

train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = True)

columns = ['neurons', 'layers', 'learn_rate', 'epochs', 'hys_score', 'hys_mse', 'hys_mape', 'jou_score', 'jou_mse', 'jou_mape', 'time']
info = pd.DataFrame(columns = columns)

In [20]:
neurons = np.arange(1, 20 + 1, 1)
layers = [1, 2]
learning_rates = [0.1, 0.01]
epochs = 10

In [21]:
model = Autoencoder(14, 2)
model

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=14, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=2, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=2, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=14, bias=True)
  )
)

In [23]:
for i in range(len(neurons)):
    for j in range(len(layers)):
        for k in range(len(learning_rates)):
            print(f"\nTraining model --- {neurons[i]}-{layers[j]}-{learning_rates[k]}-{epochs}\n")

            features_dim = len(train_data.columns.drop(target))
            out_dim = 20
            model = Autoencoder(features_dim, out_dim, neurons[i], layers[j])

            loss_func = nn.MSELoss()
            optimizer = torch.optim.SGD(model.parameters(), lr = learning_rates[k])

            for a in range(epochs):
                model.train()
                for X, y in train_loader:
                    pred_train = model(X)
                    loss = loss_func(pred_train, X)
                    
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()

time = datetime.datetime.now()
print(f"\tFinished training model at {time}.\n")

y_pred_list = []
y_test_list = []

model.eval()

with torch.no_grad():
    for X, y in test_loader:
        pred_test = model(X)
        y_pred_list.append(pred_test)
        y_test_list.append(y)

y_pred = torch.cat(y_pred_list)
y_test = torch.cat(y_test_list)

hys_score = r2_score(y_test[:, 0], y_pred[:, 0])
hys_mse = mean_squared_error(y_test[:, 0], y_pred[:, 0])
hys_mape = mean_absolute_percentage_error(y_test[:, 0], y_pred[:, 0])

jou_score = r2_score(y_test[:, 1], y_pred[:, 1])
jou_mse = mean_squared_error(y_test[:, 1], y_pred[:, 1])
jou_mape = mean_absolute_percentage_error(y_test[:, 1], y_pred[:, 1])


Training model --- 1-1-0.1-10


Training model --- 1-1-0.01-10


Training model --- 1-2-0.1-10


Training model --- 1-2-0.01-10


Training model --- 2-1-0.1-10


Training model --- 2-1-0.01-10


Training model --- 2-2-0.1-10


Training model --- 2-2-0.01-10


Training model --- 3-1-0.1-10


Training model --- 3-1-0.01-10


Training model --- 3-2-0.1-10


Training model --- 3-2-0.01-10


Training model --- 4-1-0.1-10


Training model --- 4-1-0.01-10


Training model --- 4-2-0.1-10


Training model --- 4-2-0.01-10


Training model --- 5-1-0.1-10


Training model --- 5-1-0.01-10


Training model --- 5-2-0.1-10


Training model --- 5-2-0.01-10


Training model --- 6-1-0.1-10


Training model --- 6-1-0.01-10


Training model --- 6-2-0.1-10


Training model --- 6-2-0.01-10


Training model --- 7-1-0.1-10


Training model --- 7-1-0.01-10


Training model --- 7-2-0.1-10


Training model --- 7-2-0.01-10


Training model --- 8-1-0.1-10


Training model --- 8-1-0.01-10


Training model --- 8-2-0

In [ ]:
input_dim =

autoencoder_model = Autoencoder(input_dim, latent_dim)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder_model.to(device)

print(autoencoder_model)

epochs = 100
train_losses = []
val_losses = []

for epoch in range(epochs):
    # Training
    autoencoder_model.train()
    running_train_loss = 0.0
    for data, _ in train_loader: # _ is the target, which is same as data
        data = data.to(device)
        optimizer.zero_grad()
        outputs = autoencoder_model(data)
        loss = criterion(outputs, data)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item() * data.size(0)

    epoch_train_loss = running_train_loss / len(train_loader.dataset)
    train_losses.append(epoch_train_loss)

    # Validation
    autoencoder_model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for data, _ in test_loader: # _ is the target, which is same as data
            data = data.to(device)
            outputs = autoencoder_model(data)
            loss = criterion(outputs, data)
            running_val_loss += loss.item() * data.size(0)

    epoch_val_loss = running_val_loss / len(test_loader.dataset)
    val_losses.append(epoch_val_loss)

    print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}')